In [ ]:
################################################################################
# The contents of this file are Teradata Public Content and have been released
# to the Public Domain.
# Tim Miller & Alexander Kolovos & Pankaj Vinod Purandare - Apr. 2020 - v.1.1
# Copyright (c) 2020 by Teradata
# Licensed under BSD; see "license.txt" file in the bundle root folder.
#
################################################################################
# R and Python TechBytes Demo - Part 3: teradataml
# ------------------------------------------------------------------------------
# File: R_Py_TechBytes-Part_3-Demo.py
# ------------------------------------------------------------------------------
# The R and Python TechBytes Demo comprises of 5 parts:
# Part 1 consists of only a Powerpoint overview of R and Python in Vantage
# Part 2 demonstrates the Teradata R package tdplyr for clients
# Part 3 demonstrates the Teradata Python package teradataml for clients
# Part 4 demonstrates using R in-nodes with the SCRIPT and ExecR Table Operators
# Part 5 demonstrates using Python in-nodes with the SCRIPT Table Operator
################################################################################
#
# This TechBytes demo utilizes a use case to predict the propensity of a
# financial services customer base to open a credit card account.
#
# In Section 1, Various features will be generated by joining and aggregating
# from 3 tables based tables (10K customers, 100K accounts, 1M+ transactions)
# into an analytic data set. We will show how to use teradataml functions to do
# the following:
#
#   a) Pull through the cust_id, income, age, years_with_bank, nbr_children
#      from the Customer table
#   b) Create a gender indicator variable (female_ind) from gender in the
#      Customer table
#   c) Create marital status indicator variables (single_ind, married_ind,
#      seperated_ind) from marital_status in the Customer table
#   d) Create location indicator variables (ca_resident, ny_resident,
#      tx_resident, il_resident, az_resident, oh_resident) from state_code in
#      Customer table
#   e) Create account indicator variables (ck_acct_ind, cc_acct_ind,
#      sv_acct_ind) from acct_type in the Account table
#   f) Create average balance variables (ck_avg_bal, cc_avg_bal, sv_avg_bal)
#      by taking the mean of the beginning_balance and ending_balance in the
#      Account table
#   g) Create average transaction amounts (ck_avg_tran_amt, cc_avg_tran_amt,
#      sv_avg_tran_amt) by taking the average of the principal_amt and
#      interest_amt in the Transactions table
#   h) Create quarterly transaction counts (q1_nbr_trans, q2_nbr_trans,
#      q3_nbr_trans, q4_nbr_trans) by taking the count of tran_id's based
#      upon tran_date in the Transactions table
#
# In Section 2, we create an XGBoost model and a Decision Forest model on a
# 60% sample of rows.
# Furthermore, we test/score both models with the remaining 40%.
# We conclude the present demo Part 3 with the following operations:
# - Run decision forest evaluator to determine the most pertinent variables.
# - Run confusion matrix on both scored data sets.
# - Save the models so that they can be scored again in the future.
#
# Note: Code executed successfully on Python v.3.6.8, and by using teradataml
#       v.16.20.00.05 to connect to a Vantage system that runs Advanced SQL
#       Engine database v.16.20.40.01.
################################################################################
# File Changelog
#  v.1.0     2019-10-29     First release
#  v.1.1     2020-04-02     Code simplified with case, sample teradataml funcs.
#                           Additional information about connections.
################################################################################

In [ ]:
# Load teradataml and dependency packages
from teradataml import create_context, DataFrame, get_context, copy_to_sql, in_schema
from teradataml.dataframe.sql_functions import case
from teradataml import XGBoost, XGBoostPredict, DecisionForest, DecisionForestEvaluator, DecisionForestPredict, ConfusionMatrix
from sqlalchemy.sql.expression import select, or_, extract, text, join, case as case_when
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
# Define a function that is used later in the demo code.
def draw_box_plot(data, plotColumnName, xTicksColumnName, xLabel = None, yLabel = None, title = None):
    """
    DESCRIPTION:
        Function to display bar plot using the data based on the given parameters.

    PARAMETERS:
        data:
            Required argument. A nx2 pandas dataframe with one column containing
            numeric values.

        plotColumnName:
            Required argument. Column name of the pandas dataframe that contains
            numeric values.

        xTicksColumnName:
            Required argument. Other column name for which count is given in
            plotColumnName column values.

        xLabel:
            Optional argument. Name of the label on X-axis

        yLabel:
            Optional argument. Name of the label on Y-axis

        title:
            Optional argument. Title of the bar plot

    RETURNS:
        None
    """
    index = np.arange(data.shape[0])
    fig, ax = plt.subplots(1, 1)
    ax.grid()
    ax.bar(index, data[plotColumnName])
    plt.xlabel(xLabel) if xLabel is not None else plt.xlabel(xTicksColumnName)
    plt.ylabel(yLabel) if yLabel is not None else plt.ylabel("Count")
    plt.xticks(index, data[xTicksColumnName], rotation=30)
    plt.title(title) if title is not None else plt.title("Bar Plot")
    plt.show()

## Connection

In [ ]:
# Establish connection to Teradata Vantage server (uses the Teradata SQL Driver
# for Python). Before you execute the following statement, replace the variables
# <HOSTNAME>, <UID> and <PWD> with your target Vantage system hostname (or
# IP address), and your database user ID and password, respectively.
td_context = create_context(host="<HOSTNAME>", username="<UID>", password="<PWD>")

# Notes and alternatives:
# 1. In any connection function, you can specify for an argument the getpass()
#    function of the Python standard library. First, you will need to execute:
#    import getpass
#    getpass() enables you to type your password secretly during runtime
#    without having to hard-code it in the script.
# Example: Specifying the argument password = getpass.getpass("Password: ") will
#          produce a prompt string "Password: " that expects you to type in
#          a password to proceed.
# 2. Specifying the optional argument logmech enables you to specify particular
#    logging mechanisms that may apply on your target server. For example, you
#    can connect via an active directory with LDAP credentials by specifying
#    the argument: logmech = "LDAP" as follows:
# td_context = create_context(host="<HOSTNAME>", username="<UID>", password="PWD", logmech="LDAP")

# Section 1: Data manipulation and transformations

In [ ]:
# In this section, we start with our initial input tables and use teradataml
# to perform a series of operations on the data. The goal is to produce an
# Analytic Data Set (ADS) with the features we need for the analysis.

In [ ]:
# Create DataFrames for the Customer, Accounts and Transactions tables in the
# Vantage Advanced SQL Engine. Before you execute the following statements, 
# replace the variable <DBNAME> with the target Vantage system database name
# where the corresponding table resides.
# Note: Use the in_schema() function only if tables reside in a database
#       <DBNAME> other than the default database of the connected user.
tdCustomer = DataFrame(in_schema("<DBNAME>", "Customer"))
# Using to_pandas() for a cleaner display format
tdCustomer.to_pandas().head(10)

In [ ]:
tdAccounts = DataFrame(in_schema("<DBNAME>", "Accounts"))
# Using to_pandas() for a cleaner display format
tdAccounts.to_pandas().head(10)

In [ ]:
tdTransactions = DataFrame(in_schema("<DBNAME>", "Transactions"))
# Using to_pandas() for a cleaner display format
tdTransactions.to_pandas().head(10)

## Exploratory Data Analysis: Plots

In [ ]:
### Histogram plot of income

tdCustomer_hist_pd = tdCustomer[tdCustomer.income != None].to_pandas()
# Note: to_pandas() converts the Teradata "decimal.Decimal" type into the Python
#       non-numeric "object" type. For histogram, the column should be numeric
#       type. Hence, we use astype() to convert the data type of column 'income'
#       of pandas DataFrame.
tdCustomer_hist_pd['income'] = tdCustomer_hist_pd['income'].astype('float64')
tdCustomer_hist_pd.hist(column="income", bins=30)
plt.xlabel("income")
plt.ylabel("count")
plt.title("Histogram of column 'income'")
plt.show()

In [ ]:
### Bar plot: Number of customers gender-wise

tdCustomer_bar_pd = tdCustomer.groupby("gender").count().select(['gender','count_cust_id']).to_pandas()
draw_box_plot(data = tdCustomer_bar_pd, plotColumnName = 'count_cust_id', xTicksColumnName = 'gender',
              xLabel = "Gender", yLabel = "No of customers", title = "Gender-wise customers")


In [ ]:
### Bar plot: Number of customers state-wise

tdCustomer_bar_pd = tdCustomer.groupby("state_code").count().select(['state_code','count_cust_id']).sort(['state_code']).to_pandas()
draw_box_plot(data = tdCustomer_bar_pd, plotColumnName = 'count_cust_id', xTicksColumnName = 'state_code',
              xLabel = "State ID", yLabel = "No of customers", title = "State-wise customers")


In [ ]:
### Bar plot: Number of customers Marital status-wise

tdCustomer_bar_pd = tdCustomer.groupby("marital_status").count().select(['marital_status','count_cust_id']).sort(['marital_status']).to_pandas()
draw_box_plot(data = tdCustomer_bar_pd, plotColumnName = 'count_cust_id', xTicksColumnName = 'marital_status',
              xLabel = "Marital Status", yLabel = "No of customers", title = "Marital status-wise customers")


In [ ]:
### Scatter plot: Starting and ending balance

tdAccounts_pd = tdAccounts[tdAccounts.ending_balance != None].to_pandas()
fig, ax = plt.subplots(1, 1)
ax.grid()
ax.scatter(tdAccounts_pd['starting_balance'], tdAccounts_pd['ending_balance'], s = 3)
plt.xlabel("starting_balance")
plt.ylabel("ending_balance")
plt.title("Scatter plot of starting_balance and ending_balance")
plt.show()


## Data Pre-Processing

In [ ]:
# First, grab customer demographic variables. Use imported case() function to
# create indicator variables for gender, marital_status and the state_code
# (we consider a classification into the top 6 states and the jointly the rest).
cust = tdCustomer.assign(female = case([(tdCustomer.gender == "F", 1)], else_ = 0 ),
                         single    = case( [(tdCustomer.marital_status == 1, 1)], else_ = 0 ),
                         married   = case( [(tdCustomer.marital_status == 2, 1)], else_ = 0 ),
                         separated = case( [(tdCustomer.marital_status == 3, 1)], else_ = 0 ),
                         ca_resident = case( [(tdCustomer.state_code == "CA", 1)], else_ = 0 ),
                         ny_resident = case( [(tdCustomer.state_code == "NY", 1)], else_ = 0 ),
                         tx_resident = case( [(tdCustomer.state_code == "TX", 1)], else_ = 0 ),
                         il_resident = case( [(tdCustomer.state_code == "IL", 1)], else_ = 0 ),
                         az_resident = case( [(tdCustomer.state_code == "AZ", 1)], else_ = 0 ),
                         oh_resident = case( [(tdCustomer.state_code == "OH", 1)], else_ = 0 )
                        )
cust.to_pandas().head(10)

In [ ]:
# Next: Get the account information required for the aggregation, and create
# the indicator variables for acct_type.
acct_balance = tdAccounts.starting_balance + tdAccounts.ending_balance
acct = tdAccounts.assign(ck_acct = case( [(tdAccounts.acct_type == "CK", 1)], else_ = 0 ),
                         sv_acct = case( [(tdAccounts.acct_type == "SV", 1)], else_ = 0 ),
                         cc_acct = case( [(tdAccounts.acct_type == "CC", 1)], else_ = 0 ),
                         ck_bal = case( [(tdAccounts.acct_type == "CK", acct_balance.expression)], else_ = 0 ),
                         sv_bal = case( [(tdAccounts.acct_type == "SV", acct_balance.expression)], else_ = 0 ),
                         cc_bal = case( [(tdAccounts.acct_type == "CC", acct_balance.expression)], else_ = 0 )
                        )
acct.to_pandas().head(10)

In [ ]:
# Next: Get the transaction information required for the aggregation. Pull out
# the quarter the transaction was made.
acct_mon = extract('month', tdTransactions.tran_date.expression).expression
trans = tdTransactions.assign(q1_trans = case( [(acct_mon ==  "1", 1), (acct_mon ==  "2", 1), (acct_mon ==  "3", 1)], else_ = 0 ),
                              q2_trans = case( [(acct_mon ==  "4", 1), (acct_mon ==  "5", 1), (acct_mon ==  "6", 1)], else_ = 0 ),
                              q3_trans = case( [(acct_mon ==  "7", 1), (acct_mon ==  "8", 1), (acct_mon ==  "9", 1)], else_ = 0 ),
                              q4_trans = case( [(acct_mon == "10", 1), (acct_mon == "11", 1), (acct_mon == "12", 1)], else_ = 0 ),
                             )
trans.to_pandas().head(10)

In [ ]:
### Finally, pull everything together into an analytic data set

# Initially, we wish to left join the acct and trans on 'acct_nbr' into 
# an acct_trans DataFrame.
acct_trans_cols = ['cust_id', 'acct_type', 'starting_balance', 'ending_balance',
                   'acct_acct_nbr', 'principal_amt', 'interest_amt', 'tran_id',
                   'tran_date', 'q1_trans', 'q2_trans', 'q3_trans', 'q4_trans',
                   'cc_acct', 'cc_bal', 'ck_acct', 'ck_bal', 'sv_acct', 'sv_bal']

acct_trans_tmp = acct.join(other = trans,
                           on = [acct.acct_nbr == trans.acct_nbr],
                           how = "left", lsuffix = "acct", rsuffix = "trans").select(acct_trans_cols)

# We create the target acct_trans DataFrame by also adding new columns with the
# transaction amounts for each type of account, according to the following.
acct_trans_amt = trans.principal_amt + trans.interest_amt

acct_trans = acct_trans_tmp.assign(
                 ck_tran_amt = case( [(acct_trans_tmp.acct_type == "CK", acct_trans_amt.expression)], else_ = 0 ),
                 sv_tran_amt = case( [(acct_trans_tmp.acct_type == "SV", acct_trans_amt.expression)], else_ = 0 ),
                 cc_tran_amt = case( [(acct_trans_tmp.acct_type == "CC", acct_trans_amt.expression)], else_ = 0 )
                                  )
acct_trans.to_pandas().head(10)

In [ ]:
# Then, perform a left outer join of acct_trans with cust on 'cust_id'.
# This time, obtain the resulting table by dropping all the temporary table
# columns and assigning from scratch the ones we want. For select columns,
# use the case() function to replace any None occurrences with zeros.
ADS_Py_join_tmp = cust.join(other = acct_trans,
                            on = [cust.cust_id == acct_trans.cust_id],
                            how = "left", lsuffix = "cust", rsuffix = "actr")

ADS_Py_join = ADS_Py_join_tmp.assign(drop_columns = True,
                  cust_id = ADS_Py_join_tmp.cust_cust_id,
                  income = ADS_Py_join_tmp.income,
                  age = ADS_Py_join_tmp.age,
                  gender = ADS_Py_join_tmp.gender,
                  years_with_bank = ADS_Py_join_tmp.years_with_bank,
                  nbr_children = ADS_Py_join_tmp.nbr_children,
                  marital_status = ADS_Py_join_tmp.marital_status,
                  state_code = ADS_Py_join_tmp.state_code,
                  female = ADS_Py_join_tmp.female,
                  single = ADS_Py_join_tmp.single,
                  married = ADS_Py_join_tmp.married,
                  separated = ADS_Py_join_tmp.separated,
                  ca_resident = ADS_Py_join_tmp.ca_resident,
                  ny_resident = ADS_Py_join_tmp.ny_resident,
                  tx_resident = ADS_Py_join_tmp.tx_resident,
                  il_resident = ADS_Py_join_tmp.il_resident,
                  az_resident = ADS_Py_join_tmp.az_resident,
                  oh_resident = ADS_Py_join_tmp.oh_resident,
                  acct_type = ADS_Py_join_tmp.acct_type,
                  starting_balance = ADS_Py_join_tmp.starting_balance,
                  ending_balance = ADS_Py_join_tmp.ending_balance,
                  acct_nbr = ADS_Py_join_tmp.acct_acct_nbr,
                  principal_amt = ADS_Py_join_tmp.principal_amt,
                  interest_amt = ADS_Py_join_tmp.interest_amt,
                  tran_id = ADS_Py_join_tmp.tran_id,
                  tran_date = ADS_Py_join_tmp.tran_date,
                  q1_trans = case( [(ADS_Py_join_tmp.q1_trans == None, 0)], else_ = ADS_Py_join_tmp.q1_trans ),
                  q2_trans = case( [(ADS_Py_join_tmp.q2_trans == None, 0)], else_ = ADS_Py_join_tmp.q2_trans ),
                  q3_trans = case( [(ADS_Py_join_tmp.q2_trans == None, 0)], else_ = ADS_Py_join_tmp.q2_trans ),
                  q4_trans = case( [(ADS_Py_join_tmp.q2_trans == None, 0)], else_ = ADS_Py_join_tmp.q2_trans ),
                  ck_acct = case( [(ADS_Py_join_tmp.ck_acct == None, 0)], else_ = ADS_Py_join_tmp.ck_acct ),
                  sv_acct = case( [(ADS_Py_join_tmp.sv_acct == None, 0)], else_ = ADS_Py_join_tmp.sv_acct ),
                  cc_acct = case( [(ADS_Py_join_tmp.cc_acct == None, 0)], else_ = ADS_Py_join_tmp.cc_acct ),
                  ck_bal = case( [(ADS_Py_join_tmp.ck_bal == None, 0)], else_ = ADS_Py_join_tmp.ck_bal ),
                  sv_bal = case( [(ADS_Py_join_tmp.sv_bal == None, 0)], else_ = ADS_Py_join_tmp.sv_bal ),
                  cc_bal = case( [(ADS_Py_join_tmp.cc_bal == None, 0)], else_ = ADS_Py_join_tmp.cc_bal ),
                  ck_tran_amt = case( [(ADS_Py_join_tmp.ck_tran_amt == None, 0)], else_ = ADS_Py_join_tmp.ck_tran_amt ),
                  sv_tran_amt = case( [(ADS_Py_join_tmp.sv_tran_amt == None, 0)], else_ = ADS_Py_join_tmp.sv_tran_amt ),
                  cc_tran_amt = case( [(ADS_Py_join_tmp.cc_tran_amt == None, 0)], else_ = ADS_Py_join_tmp.cc_tran_amt )
                                    )
ADS_Py_join.to_pandas().head(10)

In [ ]:
# As a last step, aggregate and roll up by 'cust_id' all variables in the above join operation.
#
# This pulls everything together into the analytic data set.
ADS_Py = ADS_Py_join.groupby("cust_id").agg(
    {
        "income" : "min",
        "age" : "min",
        "years_with_bank" : "min",
        "nbr_children" : "min",
        "female" : "min",
        "single" : "min",
        "married" : "min",
        "separated" : "min",
        "ca_resident" : "max",
        "ny_resident" : "max",
        "tx_resident" : "max",
        "il_resident" : "max",
        "az_resident" : "max",
        "oh_resident" : "max",
        "ck_acct" : "max",
        "sv_acct" : "max",
        "cc_acct" : "max",
        "ck_bal" : "mean",
        "sv_bal" : "mean",
        "cc_bal" : "mean",
        "ck_tran_amt" : "mean",
        "sv_tran_amt" : "mean",
        "cc_tran_amt" : "mean",
        "q1_trans" : "count",
        "q2_trans" : "count",
        "q3_trans" : "count",
        "q4_trans" : "count"
    }
)

# Column names have been affected by aggregation. Assign final names.
# Note: In teradataml, the rename() and column methods are unavailable yet.
#       We use the assign() method combined with select() to achieve the task.
columns = ['cust_id','tot_income','tot_age','tot_cust_years','tot_children','female_ind',
           'single_ind', 'married_ind', 'separated_ind', 'ca_resident_ind', 'ny_resident_ind',
           'tx_resident_ind','il_resident_ind','az_resident_ind', 'oh_resident_ind',
           'ck_acct_ind','sv_acct_ind','cc_acct_ind', 'ck_avg_bal','sv_avg_bal','cc_avg_bal',
           'ck_avg_tran_amt','sv_avg_tran_amt','cc_avg_tran_amt','q1_trans_cnt',
           'q2_trans_cnt','q3_trans_cnt','q4_trans_cnt']

ADS_Py = ADS_Py.assign(drop_columns = True,
                       cust_id         = ADS_Py.cust_id,
                       tot_income      = ADS_Py.min_income,
                       tot_age         = ADS_Py.min_age,
                       tot_cust_years  = ADS_Py.min_years_with_bank,
                       tot_children    = ADS_Py.min_nbr_children,
                       female_ind      = ADS_Py.min_female,
                       single_ind      = ADS_Py.min_single,
                       married_ind     = ADS_Py.min_married,
                       separated_ind   = ADS_Py.min_separated,
                       ca_resident_ind = ADS_Py.max_ca_resident,
                       ny_resident_ind = ADS_Py.max_ny_resident,
                       tx_resident_ind = ADS_Py.max_tx_resident,
                       il_resident_ind = ADS_Py.max_il_resident,
                       az_resident_ind = ADS_Py.max_az_resident,
                       oh_resident_ind = ADS_Py.max_oh_resident,
                       ck_acct_ind     = ADS_Py.max_ck_acct,
                       sv_acct_ind     = ADS_Py.max_sv_acct,
                       cc_acct_ind     = ADS_Py.max_cc_acct,
                       ck_avg_bal      = ADS_Py.mean_ck_bal,
                       sv_avg_bal      = ADS_Py.mean_sv_bal,
                       cc_avg_bal      = ADS_Py.mean_cc_bal,
                       ck_avg_tran_amt = ADS_Py.mean_ck_tran_amt,
                       sv_avg_tran_amt = ADS_Py.mean_sv_tran_amt,
                       cc_avg_tran_amt = ADS_Py.mean_cc_tran_amt,
                       q1_trans_cnt    = ADS_Py.count_q1_trans,
                       q2_trans_cnt    = ADS_Py.count_q2_trans,
                       q3_trans_cnt    = ADS_Py.count_q3_trans,
                       q4_trans_cnt    = ADS_Py.count_q4_trans).select(columns)

# teradataml DataFrame creates views at the backend which are temporary. At the
# end of the context removal, all temporary table/views perish. For this reason,
# persist the output of ADS_Py as a table in the Advanced SQL Engine. First, DROP
# the ADS_Py table, if it previously exists.
try:
    get_context().execute("DROP TABLE ADS_Py")
except:
    pass

copy_to_sql(ADS_Py, table_name="ADS_Py", if_exists="replace")

# Create a DataFrame and take a glimpse at it.
tdADS_Py = DataFrame("ADS_Py")
tdADS_Py.to_pandas().head(10)

In [ ]:
# Note: Alternatively, optimized written SQL can be used to create the analytic
#       data set, as shown in the following:
# Before you execute the following statement, replace the variable <DBNAME> with
# the target Vantage system database name where the corresponding table resides.
ADS_SQL ="SELECT \
T1.cust_id  AS cust_id \
,MIN(T1.income) AS tot_income \
,MIN(T1.age) AS tot_age \
,MIN(T1.years_with_bank) AS tot_cust_years \
,MIN(T1.nbr_children) AS tot_children \
,CASE WHEN MIN(T1.marital_status) = 1 THEN 1 ELSE 0 END AS single_ind \
,CASE WHEN MIN(T1.gender) = 'F' THEN 1 ELSE 0 END AS female_ind \
,CASE WHEN MIN(T1.marital_status) = 2 THEN 1 ELSE 0 END AS married_ind \
,CASE WHEN MIN(T1.marital_status) = 3 THEN 1 ELSE 0 END AS separated_ind \
,MAX(CASE WHEN T1.state_code = 'CA' THEN 1 ELSE 0 END) AS ca_resident_ind \
,MAX(CASE WHEN T1.state_code = 'NY' THEN 1 ELSE 0 END) AS ny_resident_ind \
,MAX(CASE WHEN T1.state_code = 'TX' THEN 1 ELSE 0 END) AS tx_resident_ind \
,MAX(CASE WHEN T1.state_code = 'IL' THEN 1 ELSE 0 END) AS il_resident_ind \
,MAX(CASE WHEN T1.state_code = 'AZ' THEN 1 ELSE 0 END) AS az_resident_ind \
,MAX(CASE WHEN T1.state_code = 'OH' THEN 1 ELSE 0 END) AS oh_resident_ind \
,MAX(CASE WHEN T2.acct_type = 'CK' THEN 1 ELSE 0 END) AS ck_acct_ind \
,MAX(CASE WHEN T2.acct_type = 'SV' THEN 1 ELSE 0 END) AS sv_acct_ind \
,MAX(CASE WHEN T2.acct_type = 'CC' THEN 1 ELSE 0 END) AS cc_acct_ind \
,AVG(CASE WHEN T2.acct_type = 'CK' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS ck_avg_bal \
,AVG(CASE WHEN T2.acct_type = 'SV' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS sv_avg_bal \
,AVG(CASE WHEN T2.acct_type = 'CC' THEN T2.starting_balance+T2.ending_balance ELSE 0 END) AS cc_avg_bal \
,AVG(CASE WHEN T2.acct_type = 'CK' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS ck_avg_tran_amt \
,AVG(CASE WHEN T2.acct_type = 'SV' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS sv_avg_tran_amt \
,AVG(CASE WHEN T2.acct_type = 'CC' THEN T3.principal_amt+T3.interest_amt ELSE 0 END) AS cc_avg_tran_amt \
,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 1 THEN T3.tran_id ELSE NULL END) AS q1_trans_cnt \
,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 2 THEN T3.tran_id ELSE NULL END) AS q2_trans_cnt \
,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 3 THEN T3.tran_id ELSE NULL END) AS q3_trans_cnt \
,COUNT(CASE WHEN ((EXTRACT(MONTH FROM T3.tran_date) + 2) / 3) = 4 THEN T3.tran_id ELSE NULL END) AS q4_trans_cnt \
FROM <DBNAME>.Customer AS T1 \
LEFT OUTER JOIN <DBNAME>.Accounts AS T2 \
ON T1.cust_id = T2.cust_id \
LEFT OUTER JOIN <DBNAME>.Transactions AS T3 \
ON T2.acct_nbr = T3.acct_nbr \
GROUP BY T1.cust_id"

# As earlier, DROP the ADS_Py table, if it previously exists.
# In this case, create a DataFrame with from_query() and take a glimpse at it.
try:
    get_context().execute("DROP TABLE ADS_SQL")
except:
    pass

tdADS_SQL = DataFrame.from_query(ADS_SQL)
tdADS_SQL.to_pandas().head(10)

# Section 2: Model fitting with the Vantage Analytic Functions

In [ ]:
# In this section, we use the ADS that we created in Section 1 above to perform
# model fitting and scoring tasks. We illustrate using teradataml functions that
# invoke the XGBoost and Decision Forest algorithms from corresponding analytic
# functions that reside in the Vantage Machine Learning Engine of the connected 
# Vantage system.
# Note: To continue into Section 2 of this demo, you must have access
#       to a Vantage system with a Machine Learning Engine component.

## Create training and testing DataSets from the persisted table 'ADS_Py'

In [ ]:
# Split the analytic data set into training and testing data sets (60/40%)
# with the sample() function.

ADS_Train_Test = tdADS_Py.sample(frac = [0.60, 0.40])

try:
    get_context().execute("DROP TABLE ADS_Train_Test")
except:
    pass

copy_to_sql(ADS_Train_Test, table_name="ADS_Train_Test", if_exists="replace")

# Create a DataFrame and take a glimpse at it.
tdTrain_Test = DataFrame("ADS_Train_Test")
tdTrain_Test.to_pandas().head(10)

In [ ]:
# Use the 60% sample to train.
TrainQuery = tdTrain_Test[tdTrain_Test.sampleid == "1"]

# Use the 40% sample to test/score.
TestQuery = tdTrain_Test[tdTrain_Test.sampleid == "2"]

## Model training and scoring using XGBoost

In [ ]:
# First, construct a formula to predict Credit Card account ownership based on independent variables of interest:
formula = "cc_acct_ind ~ tot_income + tot_age + tot_cust_years + tot_children + female_ind + single_ind " \
          "+ married_ind + separated_ind + ca_resident_ind + ny_resident_ind + tx_resident_ind + il_resident_ind " \
          "+ az_resident_ind + oh_resident_ind + ck_acct_ind + sv_acct_ind + ck_avg_bal + sv_avg_bal " \
          "+ ck_avg_tran_amt + sv_avg_tran_amt"

In [ ]:
# Then, train an XGBoost model to predict Credit Card account ownership on the basis of the above formula.
td_xgboost_model = XGBoost(data=TrainQuery,
                           id_column='cust_id',
                           formula = formula,
                           num_boosted_trees=4,
                           loss_function='binomial',
                           prediction_type='classification',
                           reg_lambda=1.0,
                           shrinkage_factor=0.1,
                           iter_num=10,
                           min_node_size=1,
                           max_depth=10
                           )
print(td_xgboost_model)

In [ ]:
# Score the XGBoost model against the holdout and compare actuals to predicted.
td_xgboost_predict = XGBoostPredict(td_xgboost_model,
                                    newdata=TestQuery,
                                    object_order_column= ['tree_id','iter','class_num'],
                                    id_column='cust_id',
                                    terms='cc_acct_ind',
                                    num_boosted_trees=4
                                    )

# Persist the XGBoostPredict output.
try:
    get_context().execute("DROP TABLE XGBoost_Scores")
except:
    pass

td_xgboost_predict.result.to_sql(if_exists = "replace", table_name = "XGBoost_Scores")
tdXGBoost_Scores = DataFrame("XGBoost_Scores")
tdXGBoost_Scores.to_pandas().head(10)

## Model training and scoring using Random Forest

In [ ]:
# In a different approach, train a Random Forest model to predict the same target,
# so we can compare and see which algorithm fits best the data.
# We implement this approach by means of the Decision Forest functions.
td_decisionforest_model = DecisionForest(formula = formula,
                                         data = TrainQuery,
                                         tree_type = "classification",
                                         ntree = 500,
                                         nodesize = 1,
                                         variance = 0.0,
                                         max_depth = 12,
                                         mtry = 5,
                                         mtry_seed = 100,
                                         seed = 100
                                         )
print(td_decisionforest_model)

In [ ]:
# Call the DecisionForestEvaluator() function to determine the most important
# variables in the Decision Forest model.
td_decisionforest_model_evaluator = DecisionForestEvaluator(object = td_decisionforest_model,
                                                            num_levels = 5)
# In the following, the describe() method provides summary statistics across
# the trees over grouping by each variable. One can consider the mean
# importance across all trees as the importance for each variable.
td_variable_importance = td_decisionforest_model_evaluator.result.select(["variable_col", "importance"]).groupby("variable_col").describe()
print(td_variable_importance)

In [ ]:
# Score the Decision Forest model.
td_decisionforest_predict = DecisionForestPredict(td_decisionforest_model,
                                                  newdata = TestQuery,
                                                  id_column = "cust_id",
                                                  detailed = False,
                                                  terms = ["cc_acct_ind"]
                                                  )

# Persist the DecisionForestPredict output.
try:
    get_context().execute("DROP TABLE RandomForest_Scores")
except:
    pass

copy_to_sql(td_decisionforest_predict.result, if_exists = "replace", table_name="RandomForest_Scores")
tdRandomForest_Scores = DataFrame("RandomForest_Scores")
tdRandomForest_Scores.to_pandas().head(10)

## Using Confusion Matrix to look at the 2 models

In [ ]:
# Look at the confusion matrix for the XGBoost model.
confusion_matrix_XGB = ConfusionMatrix(data = tdXGBoost_Scores,
                                       reference = "cc_acct_ind",
                                       prediction = "prediction"
                                       )
print(confusion_matrix_XGB)

In [ ]:
# Look at the confusion matrix for Random Forest model.
confusion_matrix_DF = ConfusionMatrix(data = tdRandomForest_Scores,
                                      reference = "cc_acct_ind",
                                      prediction = "prediction"
                                      )
print(confusion_matrix_DF)

## Saving models to be reused

In [ ]:
# Note: Model Administration APIs are not officially supported in teradataml v.16.20.00.05
#       and their APIs are not exposed to end user.
from teradataml import save_model, list_models, describe_model, delete_model, retrieve_model

In [ ]:
# Save the models so that they can be scored again and managed moving forward.
# Expect an error if you ask to save a model with an existing specified name.
save_model(td_xgboost_model, name="XGBoost_Model_1")
save_model(td_decisionforest_model, name="Decision_Forest_Model_1")

In [ ]:
### Show some other methods for model maintenance

# List existing saved models.
list_models()

In [ ]:
# Provide description of a specified model.
# Expect an error if a specified model does not exist.
describe_model(name = "Decision_Forest_Model_1")

In [ ]:
# Retrieve saved models and use these in a different session to score.
# Expect an error if a specified model does not exist.
xgboost_model = retrieve_model(name = "XGBoost_Model_1")
decision_forest_model = retrieve_model(name="Decision_Forest_Model_1")

In [ ]:
# Delete a saved model.
# Expect an error if a specified model does not exist.
delete_model(name="XGBoost_Model_1")
delete_model(name="Decision_Forest_Model_1")
list_models()

## End of session

In [ ]:
# Remove the context of present teradataml session and terminate the Python session.
# It is recommended to call the remove_context() function for session cleanup.
# Temporary objects are removed at the end of the session.
from teradataml import remove_context
remove_context()